In [ ]:
!nvidia-smi

In [ ]:
#!pip install -q tensorflow_privacy

In [ ]:
!pip install opacus

In [ ]:
!ls "../"

In [ ]:
from google.colab import drive

drive.mount('/gdrive', force_remount = True)

In [ ]:
!gsutil cp -r "/gdrive/My Drive/SVHN3_1.zip" "../SVHN3_1.zip"

In [ ]:
!unzip "../SVHN3_1.zip" -d "../"

In [ ]:
!ls -lrt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from tqdm import tqdm as tqdm
import pandas as pd

import torchvision
import random
import os
import glob
import copy
from torchsummary import summary
from PIL import Image

import opacus

import matplotlib.pyplot as plt

seed = 42
best_model = None
best_loss = 0.
best_test_loss = 0.
best_test_acc = 0.
best_pred_labels = []
true_labels = []

pred_labels = []
test_acc = 0.
test_loss = 0.

# device = torch.device('cuda:0')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# PREPROCESS

In [ ]:
new_dict = {}

In [ ]:
l = ["Paths", "Labels"]

In [ ]:
new_dict = dict(zip(new_dict, l))
new_dict

In [ ]:
new_dict.update(dict(zip(new_dict, l)))
new_dict

In [ ]:
pathlist = []
label = []

for file in os.listdir("../SVHN3_1/train/"):
  for n in os.listdir("../SVHN3_1/train/"+file):
    pathlist.append("../SVHN3_1/train/"+file+"/"+n)
    label.append(int(file))    

In [ ]:
len(label)

In [ ]:
len(pathlist)

In [ ]:
data = {
   'Paths': [],
   'Labels': []
}
df = pd.DataFrame(data)

df['Paths'] = pathlist
df['Labels'] = label

df.head(5)

In [ ]:
len(df)

In [ ]:
from google.colab import files

df.to_csv('../gdrive/My Drive/SVHN3_1_train.csv')
df.to_csv('../SVHN3_1_train.csv')
#files.download('../gdrive/My Drive/.csv')

In [ ]:
new_dict = {}

In [ ]:
l = ["Paths", "Labels"]

In [ ]:
new_dict = dict(zip(new_dict, l))
new_dict

In [ ]:
new_dict.update(dict(zip(new_dict, l)))
new_dict

In [ ]:
pathlist = []
label = []

for file in os.listdir("../SVHN3_1/test/"):
  for n in os.listdir("../SVHN3_1/test/"+file):
    pathlist.append("../SVHN3_1/test/"+file+"/"+n)
    label.append(int(file))    

In [ ]:

len(label)

In [ ]:
len(pathlist)

In [ ]:
data = {
   'Paths': [],
   'Labels': []
}
df = pd.DataFrame(data)

df['Paths'] = pathlist
df['Labels'] = label

df.head(5)

In [ ]:
len(df)

In [ ]:
from google.colab import files

df.to_csv('../gdrive/My Drive/SVHN3_1_test.csv')
df.to_csv('../SVHN3_1_test.csv')
#files.download('../gdrive/My Drive/.csv')

In [ ]:
class CustomDatasetFromCsvData(Dataset):
    def __init__(self, csv_path, transform=None):
        """
        Custom dataset example for reading data from csv
        Args:
            csv_path (string): path to csv file
            height (int): image height
            width (int): image width
            transform: pytorch transforms for transforms and tensor conversion
        """
        self.data = pd.read_csv(csv_path)
        self.paths = self.data['Paths']
        self.labels = np.asarray(self.data['Labels'])
        #self.height = height
        #self.width = width
        self.transform = transform

    def __getitem__(self, index):
        single_image_label = self.labels[index]
        # Read each 784 pixels and reshape the 1D array ([784]) to 2D array ([28,28])
        img_as_np = Image.open(self.paths[index])
        img_as_np = np.reshape(img_as_np, (32, 32, 3))
        #img_as_np = np.asarray(self.data.iloc[index][1:]).reshape(28, 28).astype('uint8')
        # Convert image from numpy array to PIL image, mode 'L' is for grayscale
        img_as_img = Image.fromarray(img_as_np)
        #img_as_img = img_as_img.convert('L')
        # Transform image to tensor
        if self.transform is not None:
            img_as_tensor = self.transform(img_as_img)
        #img_as_tensor = Variable(img_as_tensor)
        #single_image_label = Variable(torch.Tensor(single_image_label))
        # Return image and the label
        return (img_as_tensor, single_image_label)

    def __len__(self):
        return len(self.data.index)

In [ ]:
train_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])
test_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])

In [ ]:
from opacus.utils.uniform_sampler import UniformWithReplacementSampler

PATH_TO_SVHN3 = '../SVHN3_1'

trainset = datasets.ImageFolder(root=PATH_TO_SVHN3+'/train',transform=train_transform)
testset = datasets.ImageFolder(root=PATH_TO_SVHN3+'/test',transform=test_transform)

BATCH_SIZE=128

SAMPLE_RATE = BATCH_SIZE / len(trainset)

trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_sampler=UniformWithReplacementSampler(
        num_samples=len(trainset),
        sample_rate=SAMPLE_RATE,
    ),
)

testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=128,
    shuffle=False
)

In [ ]:
# from opacus.dp_model_inspector import DPModelInspector

# inspector = DPModelInspector()
# inspector.validate(model)

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(type(images))
print(type(labels))
print(images.shape)
print(labels.shape)

In [ ]:
plt.imshow(images[10].permute(1,2,0).numpy());

In [ ]:
import matplotlib.pyplot as plt
import collections

class_freq_counter = collections.Counter(trainloader.dataset.targets)
train_class_freq = [val for val in class_freq_counter.values()]

plt.figure(figsize = (10,10))
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
for idx, val in enumerate(train_class_freq):
  plt.text((idx-0.25),val+40, str(val), fontsize=12)
plt.bar(classes, train_class_freq, color='green')
plt.title("SVHN3 Training Dataset Distribution")

In [ ]:
train_class_freq

In [ ]:
print("Total number of samples in Train dataset :", np.sum(train_class_freq))

In [ ]:
import matplotlib.pyplot as plt
import collections

class_freq_counter = collections.Counter(testloader.dataset.targets)
test_class_freq = [val for val in class_freq_counter.values()]

plt.figure(figsize = (10,10))
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
for idx, val in enumerate(test_class_freq):
  plt.text((idx-0.25),val+10, str(val), fontsize=12)
#plt.text(0,980, "980")
#plt.text(1,1135, "1135")
plt.bar(classes, test_class_freq, color='green')
plt.title("SVHN3 Test Dataset Distribution")
plt.show()

In [ ]:
test_class_freq

In [ ]:
print("Total number of samples in Train dataset :", np.sum(test_class_freq))

In [ ]:
'''

class Model(nn.Module):
    """Class used to initialize model of student/teacher"""

    def __init__(self):

        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #return F.log_softmax(x, dim=1)
        return x

'''

In [ ]:


# class Model(nn.Module):
#     """Class used to initialize model of student/teacher"""
#     def __init__(self):
#         super(Model, self).__init__()
#         self.conv1 = nn.Conv2d(3, 96, 3, 1)
#         self.conv2 = nn.Conv2d(96, 96, 3, 1)
#         self.conv3 = nn.Conv2d(96, 96, 3, 1)
#         #self.bn1 = nn.BatchNorm2d(96)
#         self.conv4 = nn.Conv2d(96, 192, 3, 1)
#         self.conv5 = nn.Conv2d(192, 192, 3, 1)
#         self.conv6 = nn.Conv2d(192, 192, 3, 1)
#         self.conv7 = nn.Conv2d(192, 192, 5, 1)
#         #self.bn2 = nn.BatchNorm2d(192)
#         self.fc1 = nn.Linear(2 * 2 * 192, 500)
#         self.fc2 = nn.Linear(500, 10)
#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = F.max_pool2d(x, 3, 1)
#         #x = self.bn1(x)
#         x = F.dropout(x, 0.5)
#         x = F.relu(self.conv2(x))
#         x = F.max_pool2d(x, 3, 1)
#         #x = self.bn1(x)        
#         x = F.dropout(x, 0.5)
#         x = F.relu(self.conv3(x))
#         x = F.max_pool2d(x, 3, 1)
#         #x = self.bn1(x)
#         x = F.dropout(x, 0.5)
#         x = F.relu(self.conv4(x))
#         x = F.max_pool2d(x, 3, 1)
#         #x = self.bn2(x)
#         x = F.dropout(x, 0.5)
#         x = F.relu(self.conv5(x))
#         x = F.max_pool2d(x, 3, 1)
#         #x = self.bn2(x)
#         x = F.dropout(x, 0.5)
#         x = F.relu(self.conv6(x))
#         x = F.max_pool2d(x, 3, 1)
#         #x = self.bn2(x)
#         x = F.dropout(x, 0.5)
#         x = F.relu(self.conv7(x))
#         x = F.max_pool2d(x, 3, 1)
#         #x = self.bn2(x)
#         x = F.dropout(x, 0.5)
#         # print(x.shape)
#         x = x.view(-1, 2 * 2 * 192)
#         # print(x.shape)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         #return F.log_softmax(x, dim=1)
#         return x

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

In [ ]:
model = ResNet18()

In [ ]:
summary(model.cuda(), (3,32,32))

In [ ]:
from opacus.dp_model_inspector import DPModelInspector

inspector = DPModelInspector()
inspector.validate(model)

In [ ]:
from opacus.dp_model_inspector import DPModelInspector
from opacus.utils import module_modification

model = module_modification.convert_batchnorm_modules(model)
inspector = DPModelInspector()
print(f"Is the model valid? {inspector.validate(model)}")

In [ ]:
model = model.cuda()

In [ ]:
def accuracy(predictions, dataset):
    """Evaluates accuracy for given set of predictions and true labels.
       Args:
           predictions[torch tensor]: predictions made by classifier.
           labels[torch tensor]: true labels of the dataset.
       Returns:
           accuracy[float]: accuracy of classifier.
    """

    total = 0.0
    correct = 0.0

    for j in range(0, len(dataset)):
        correct += (predictions[j].cpu().long() == dataset[j].cpu().long()).sum().item()
        total += len(dataset[j])

    return (correct / total) * 100

In [ ]:
#model = Model().cuda()

### DP-SGD with eps = 5.06

In [ ]:
# batchsize = 256
# from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

# #from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent


# rdp = compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=np.sum(train_class_freq), batch_size=batchsize, noise_multiplier=0.66, epochs=18, delta=1e-4)
# #epsilon = get_privacy_spent(5218, rdp, target_delta=1e-5)[0]
# rdp

In [ ]:
# batchsize = 32
# from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

# #from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent


# rdp = compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=np.sum(train_class_freq), batch_size=batchsize, noise_multiplier=0.51, epochs=18, delta=1e-4)
# #epsilon = get_privacy_spent(5218, rdp, target_delta=1e-5)[0]
# rdp

In [ ]:
# train_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
# test_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

In [ ]:
# PATH_TO_SVHN3 = '/content/SVHN3_1'

# trainset = datasets.ImageFolder(root=PATH_TO_SVHN3+'/train',transform=train_transform)
# testset = datasets.ImageFolder(root=PATH_TO_SVHN3+'/test',transform=test_transform)
# train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.5)

In [ ]:
def accuracy(preds, labels):
    return (preds == labels).mean()

In [ ]:
EPOCHS=50
EPSILON=5
MAX_GRAD_NORM = 1
DELTA = 1e-5
VIRTUAL_BATCH_SIZE=256
N_ACCUMULATION_STEPS = int(VIRTUAL_BATCH_SIZE / BATCH_SIZE)

In [ ]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine(
    model,
    sample_rate=SAMPLE_RATE * N_ACCUMULATION_STEPS,
    epochs = EPOCHS,
    target_epsilon = EPSILON,
    #noise_multiplier = 0.8591845703125,
    target_delta = DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)
privacy_engine.attach(optimizer)

print(f"Using sigma={privacy_engine.noise_multiplier} and C={MAX_GRAD_NORM}")

In [ ]:
import numpy as np

def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []

    for i, (data, target) in enumerate(train_loader):  


        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)

        preds = np.argmax(output.detach().cpu().numpy(), axis=1)
        labels = target.detach().cpu().numpy()
        
        # measure accuracy and record loss
        acc = accuracy(preds, labels)
        losses.append(loss.item())
        top1_acc.append(acc) 
        #print(loss)
        #print(acc)       

        loss.backward()
        	
        # take a real optimizer step after N_VIRTUAL_STEP steps t
        if ((i + 1) % N_ACCUMULATION_STEPS == 0) or ((i + 1) == len(train_loader)):
            optimizer.step()
        else:
            optimizer.virtual_step() # take a virtual step

        if i % 200 == 0:
            epsilon, best_alpha = optimizer.privacy_engine.get_privacy_spent(DELTA)
            print(
                f"\tTrain Epoch: {epoch} \t"
                f"Loss: {np.mean(losses):.6f} "
                f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
                f"(ε = {epsilon:.2f}, δ = {DELTA})"
            )

In [ ]:
def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)

    print(
        f"\tTest set:"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)

In [ ]:
def test1(model, testloader, optimizer):
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    model.eval()
    #correct = 0
    #total = 0

    epoch_loss = 0.
    epoch_acc = 0.
    
    batch_num = 0.
    samples_num = 0.
    
    true_labels = []
    pred_labels = []

    with torch.no_grad():
        for batch_idx, data in  enumerate(testloader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            #_, predicted = torch.max(outputs.data, 1)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            true_labels.append(labels.detach().cpu())
            pred_labels.append(preds.detach().cpu())

            # print(f'\r{phase} batch [{batch_idx}/{len(testloader)}]: loss {loss}', end='', flush=True)
            #print(f'\r{phase} batch [{batch_idx}/{len(testloader)}]: loss {torch.mean(loss).item()}', end='', flush=True)
            epoch_loss += torch.mean(loss.detach().cpu()).item()
            epoch_acc += torch.sum(preds == labels.data)
            batch_num += 1
            samples_num += len(labels)

            #total += labels.size(0)
            #correct += (predicted == labels).sum().item()
        #print("TESTING.....")
        return epoch_loss / batch_num, epoch_acc / samples_num, torch.cat(true_labels).numpy(), torch.cat(pred_labels).numpy()

In [ ]:
device

In [ ]:
trainloader

In [ ]:
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

for i in range(1):

  #model = ResNet18()

  #model = model.cuda()

  #model = module_modification.convert_batchnorm_modules(model)

  for epoch in tqdm(range(EPOCHS)):
    train(model, trainloader, optimizer, epoch + 1, device)



  top1_acc = test(model, testloader, device)

  Test_loss_after_training, test_acc_after_training, true_labels_test, pred_labels_test = test1(model, testloader, optimizer)


  conf_mat_test =confusion_matrix(true_labels_test, pred_labels_test)
  class_accuracy_test=100*conf_mat_test.diagonal()/conf_mat_test.sum(1)

  print("Test results for run {}".format(i+1))
  for label, acc in enumerate(class_accuracy_test):
    print("Test accuracy of class {} is {}".format(str(label), str(round(acc, 2))+"%"))

In [ ]:
top1_acc = test(model, testloader, device)

In [ ]:
Test_loss_after_training, test_acc_after_training, true_labels_test, pred_labels_test = test(model, testloader, optimizer)
#test_nrun_true_labels.append(true_labels_test)
#test_nrun_pred_labels.append(pred_labels_test)
#test_nrun_acc.append(test_acc_after_training)

In [ ]:
from sklearn.metrics import confusion_matrix



conf_mat_test =confusion_matrix(true_labels_test, pred_labels_test)
class_accuracy_test=100*conf_mat_test.diagonal()/conf_mat_test.sum(1)
print("Test results for run 1")
print(class_accuracy_test)
for label, acc in enumerate(class_accuracy_test):
  print("Test accuracy of class {} is {}".format(str(label), str(round(acc, 2))+"%"))

# IGNORE BELOW CODE

In [ ]:
def clip_grad(parameters, max_norm, norm_type=2):
    parameters = list(filter(lambda p: p.grad is not None, parameters))
    total_norm = 0
    for p in parameters:
        param_norm = p.grad.data.norm(norm_type)
        total_norm += param_norm.item() ** norm_type

In [ ]:
def train_dp(model, trainloader, criterion, optimizer, lr_scheduler, phase='train'):
    """
    Differentially Private version of the training procedure
    :param trainloader:
    :param model:
    :param optimizer:
    :param epoch:
    :return:
    """
    model.train()
    running_loss = 0.0
    epoch_loss = 0.
    epoch_acc = 0.
    
    batch_num = 0.
    samples_num = 0.
    
    true_labels = []
    pred_labels = []
    #num_microbatches = 32*2
    num_microbatches = 32
    S = 1
    '''
    z = 0.77  #DP-SGD with sampling rate = 0.543% and noise_multiplier = 0.77 iterated over 9201 steps satisfies differential privacy with eps = 5.06 and delta = 0.0001.
              #The optimal RDP order is 4.0.
    '''

    z = 0.66  #DP-SGD with sampling rate = 0.543% and noise_multiplier = 0.66 iterated over 3313 steps satisfies differential privacy with eps = 4.99 and delta = 0.0001.
              #The optimal RDP order is 3.5.
    sigma = z * S
    for batch_idx, data in tqdm(enumerate(trainloader, 0), leave=True):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        model = model.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        print(loss)
        print(loss.shape)
        running_loss += torch.mean(loss).item()

        true_labels.append(labels.detach().cpu())
        pred_labels.append(preds.detach().cpu())

        if loss.shape[0] == 3:
          losses = torch.mean(loss.reshape(3, -1), dim=1)
        else:
          losses = torch.mean(loss.reshape(num_microbatches, -1), dim=1)
        
        saved_var = dict()
        for tensor_name, tensor in model.named_parameters():
            saved_var[tensor_name] = torch.zeros_like(tensor)

        for j in losses:
            j.backward(retain_graph=True)
            torch.nn.utils.clip_grad_norm_(model.parameters(), S)
            for tensor_name, tensor in model.named_parameters():
                new_grad = tensor.grad
                saved_var[tensor_name].add_(new_grad)
            model.zero_grad()

        for tensor_name, tensor in model.named_parameters():
            if device.type =='cuda':
                noise = torch.cuda.FloatTensor(tensor.grad.shape).normal_(0, sigma)
                #noise = torch.FloatTensor(tensor.grad.shape).normal_(0, sigma)
            else:
                noise = torch.FloatTensor(tensor.grad.shape).normal_(0, sigma)
            saved_var[tensor_name].add_(noise)
            tensor.grad = saved_var[tensor_name] / num_microbatches
        optimizer.step()

        #print(f'\r{phase} batch [{batch_idx}/{len(trainloader)}]: loss {loss}', end='', flush=True)
        ##print(f'\r{phase} batch [{batch_idx}/{len(trainloader)}]: loss {torch.mean(loss).item()}', end='', flush=True)
        epoch_loss += torch.mean(loss.detach().cpu()).item()
        epoch_acc += torch.sum(preds == labels.data)
        batch_num += 1
        samples_num += len(labels)

        '''if i > 0 and i % 20 == 0:
            #             logger.info('[%d, %5d] loss: %.3f' %
            #                   (epoch + 1, i + 1, running_loss / 2000))
            plot(epoch * len(trainloader) + i, running_loss, 'Train Loss')
            running_loss = 0.0'''
    #print("RETURNING........")
    return epoch_loss / batch_num, epoch_acc / samples_num, torch.cat(true_labels).numpy(), torch.cat(pred_labels).numpy()

In [ ]:
def test(model, testloader, criterion, optimizer, lr_scheduler, phase='test'):
    model = model.to(device)
    model.eval()
    #correct = 0
    #total = 0

    epoch_loss = 0.
    epoch_acc = 0.
    
    batch_num = 0.
    samples_num = 0.
    
    true_labels = []
    pred_labels = []

    with torch.no_grad():
        for batch_idx, data in  enumerate(testloader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            #_, predicted = torch.max(outputs.data, 1)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            true_labels.append(labels.detach().cpu())
            pred_labels.append(preds.detach().cpu())

            # print(f'\r{phase} batch [{batch_idx}/{len(testloader)}]: loss {loss}', end='', flush=True)
            #print(f'\r{phase} batch [{batch_idx}/{len(testloader)}]: loss {torch.mean(loss).item()}', end='', flush=True)
            epoch_loss += torch.mean(loss.detach().cpu()).item()
            epoch_acc += torch.sum(preds == labels.data)
            batch_num += 1
            samples_num += len(labels)

            #total += labels.size(0)
            #correct += (predicted == labels).sum().item()
        #print("TESTING.....")
        return epoch_loss / batch_num, epoch_acc / samples_num, torch.cat(true_labels).numpy(), torch.cat(pred_labels).numpy()

In [ ]:
criterion=nn.CrossEntropyLoss(reduction="none")
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=0.001, momentum=0.5)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.5)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

train_nrun_losses = []
val_nrun_losses = []
test_nrun_losses = []
test_nrun_true_labels = []
test_nrun_pred_labels = []
test_nrun_acc = []

def train_n(runs, nepochs = 50):
  for run in range(runs):
    model = Model().cuda()
    criterion = torch.nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=0.001)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
    print("Train Log of run ", run)
    train_losses = []
    val_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    best_model = None
    for epoch in range(nepochs):
        print('='*15, f'Epoch: {epoch}')
    
        #train_dp(model, trainloader, criterion, optimizer, lr_scheduler, phase='train')
        #test(model, testloader, criterion, optimizer, lr_scheduler, phase='test')

        train_loss, train_acc, true_labels_train, pred_labels_train = train_dp(model, train_loader, criterion, optimizer, lr_scheduler)
        #val_loss, val_acc, _, _ = test(model, test_loader, criterion, optimizer, lr_scheduler, phase='val')
        test_loss, test_acc, _, _ = test(model, test_loader, criterion, optimizer, lr_scheduler, phase='test')
        #test_loss, test_acc, true_labels, pred_labels = run_epoch(model, test_dataloader, criterion, optimizer, lr_scheduler, phase='test')
    
        print()
        print(f'Train loss: {train_loss}, Train accuracy: {train_acc}')
        #print(f'Val loss: {val_loss}, Val accuracy: {val_acc}')
        print(f'Test loss: {test_loss}, Test accuracy: {test_acc}')
        print()
    
        ##train_losses.append(train_loss)
        ##train_accuracies.append(train_acc)
        #val_losses.append(test_loss)
        #test_losses.append(test_loss)
        ##test_accuracies.append(test_acc)


        
        conf_mat_train =confusion_matrix(true_labels_train, pred_labels_train)
        class_accuracy_train=100*conf_mat_train.diagonal()/conf_mat_train.sum(1)
        print("Train results for epoch ", epoch)
        for label_t, acc_t in enumerate(class_accuracy_train):
          print("Train accuracy of class {} is {}".format(str(label_t), str(round(acc_t, 2))+"%"))

        Test_loss_after_training, test_acc_after_training, true_labels_test, pred_labels_test = test(model, test_loader, criterion, optimizer, lr_scheduler, phase='test')
        conf_mat_test =confusion_matrix(true_labels_test, pred_labels_test)
        class_accuracy_test=100*conf_mat_test.diagonal()/conf_mat_test.sum(1)
        print("Test results for epoch ", epoch)
        for label, acc in enumerate(class_accuracy_test):
          print("Test accuracy of class {} is {}".format(str(label), str(round(acc, 2))+"%"))
    
        torch.save({'epoch': epoch, 'model': model.state_dict()}, f'covnet-{seed}.pt')
    
        '''
        if best_model is None or test_loss < best_loss:
            best_model = copy.deepcopy(model)
            #best_loss = val_loss
            best_test_loss = test_loss
            best_test_acc = test_acc 
            best_pred_labels = pred_labels
            torch.save({'epoch': epoch, 'model': model.state_dict()}, f'covnet-{seed}.pt')
        '''
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    test_loss, test_acc, _, _ = test(model, test_loader, criterion, optimizer, lr_scheduler, phase='test')
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

    train_nrun_losses.append(train_losses)
    test_nrun_losses.append(test_losses)
    Test_loss_after_training, test_acc_after_training, true_labels_test, pred_labels_test = test(model, test_loader, criterion, optimizer, lr_scheduler, phase='test')
    test_nrun_true_labels.append(true_labels_test)
    test_nrun_pred_labels.append(pred_labels_test)
    test_nrun_acc.append(test_acc_after_training)
    conf_mat_test =confusion_matrix(true_labels_test, pred_labels_test)
    class_accuracy_test=100*conf_mat_test.diagonal()/conf_mat_test.sum(1)
    print("Test results for run ", run)
    for label, acc in enumerate(class_accuracy_test):
      print("Test accuracy of class {} is {}".format(str(label), str(round(acc, 2))+"%"))


In [ ]:
train_n(1)

In [ ]:
'''

train_losses = []
val_losses = []
test_losses = []

for epoch in range(50):
    print('='*15, f'Epoch: {epoch}')
    
#train_dp(model, trainloader, criterion, optimizer, lr_scheduler, phase='train')
#test(model, testloader, criterion, optimizer, lr_scheduler, phase='test')

    train_loss, train_acc, _, _ = train_dp(model, train_loader, criterion, optimizer, lr_scheduler)
    #val_loss, val_acc, _, _ = test(model, test_loader, criterion, optimizer, lr_scheduler, phase='val')
    test_loss, test_acc, _, _ = test(model, test_loader, criterion, optimizer, lr_scheduler, phase='val') ### IMPORTANT :::: Although this is test_dataloader, the name for phase is given as 'val', it is just a incorrect label and nothing else
    #test_loss, test_acc, true_labels, pred_labels = run_epoch(model, test_dataloader, criterion, optimizer, lr_scheduler, phase='test')
    
    print()
    print(f'Train loss: {train_loss}, Train accuracy: {train_acc}')
    #print(f'Val loss: {val_loss}, Val accuracy: {val_acc}')
    print(f'Test loss: {test_loss}, Test accuracy: {test_acc}')
    print()
    
    train_losses.append(train_loss)
    #val_losses.append(test_loss)
    test_losses.append(test_loss)
    
    torch.save({'epoch': epoch, 'model': model.state_dict()}, f'covnet-{seed}.pt')
    
    if best_model is None or test_loss < best_loss:
        best_model = copy.deepcopy(model)
        #best_loss = val_loss
        best_test_loss = test_loss
        best_test_acc = test_acc 
        best_pred_labels = pred_labels
        torch.save({'epoch': epoch, 'model': model.state_dict()}, f'covnet-{seed}.pt')

'''

In [ ]:
'''
train_after_loss, train_after_acc, true_labels_train, pred_labels_train = test(model, train_loader, criterion, optimizer, lr_scheduler, phase='test')
print(f'Train loss (After Training is Completed): {train_after_loss}, Train accuracy (After Training is Completed): {train_after_acc}')
'''

In [ ]:
'''
test_nrun_true_labels = []
test_nrun_pred_labels = []
test_nrun_acc = []
'''

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

runs = 5
class_accuracy_nrun_test = []
for run in range(runs):
  conf_mat_test =confusion_matrix(test_nrun_true_labels[run], test_nrun_pred_labels[run])
  class_accuracy_test=100*conf_mat_test.diagonal()/conf_mat_test.sum(1)
  #print(class_accuracy_test)
  class_accuracy_nrun_test.append(class_accuracy_test)

mean_class_acc_test = np.mean(np.array(class_accuracy_nrun_test), axis=0)
std_class_acc_test = np.std(np.array(class_accuracy_nrun_test), axis=0)

In [ ]:
for label, acc in enumerate(mean_class_acc_test):
  print("Average Test accuracy of class {} is {}".format(str(label), str(round(acc, 2))+"%"))

In [ ]:
for label, std_acc in enumerate(std_class_acc_test):
  print("Standard Deviation of Test accuracy of class {} is {}".format(str(label), str(round(std_acc, 2))+"%"))

In [ ]:
import matplotlib.pyplot as plt


labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

width = 0.35       # the width of the bars: can also be len(x) sequence

fig, ax = plt.subplots(figsize = (10,10))

ax.bar(labels, mean_class_acc_test, width, yerr=std_class_acc_test, label='SVHN Digits(Test)')

ax.set_ylabel('Class-Wise Accuracies')
ax.set_title('Class-Wise Test Accuracies for 5 runs (EPS = 5.0)')
ax.legend()

plt.show()